In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
import shutil

TensorFlow version: 2.11.0
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [32]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.05
EPOCHS = 50

In [33]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [34]:
X_train.shape

(436, 8)

In [35]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [36]:
class SomeModel(Model):
  def __init__(self, neurons_cnt=128):
    super(SomeModel, self).__init__()

    self.d_in = Dense(8, activation='relu')
    self.d1 = Dense(neurons_cnt, activation='relu')
    self.d2 = Dense(neurons_cnt, activation='relu')
    self.d_out = Dense(1)

  def call(self, x):
    x = self.d_in(x)
    x = self.d1(x)
    return self.d_out(x)

In [37]:
# Create an instance of the model
model = SomeModel()

In [38]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [39]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [40]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [41]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [42]:
from tensorflow import keras

In [43]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir
  )

Epoch 1, Loss: 1836.6314697265625, Accuracy: 26.772335052490234, Test Loss: 186.45669555664062, Test MAE: 13.622997283935547
Epoch 2, Loss: 63.46037673950195, Accuracy: 6.925711631774902, Test Loss: 32.88878631591797, Test MAE: 5.6812944412231445
Epoch 3, Loss: 20.731775283813477, Accuracy: 3.4891653060913086, Test Loss: 25.660781860351562, Test MAE: 4.990583419799805
Epoch 4, Loss: 25.706134796142578, Accuracy: 4.84944486618042, Test Loss: 12.958181381225586, Test MAE: 3.509610176086426
Epoch 5, Loss: 16.78478240966797, Accuracy: 3.942241668701172, Test Loss: 11.663522720336914, Test MAE: 3.307110071182251
Epoch 6, Loss: 9.935452461242676, Accuracy: 3.0274620056152344, Test Loss: 8.045533180236816, Test MAE: 2.7192118167877197
Epoch 7, Loss: 8.748234748840332, Accuracy: 2.8353326320648193, Test Loss: 10.110368728637695, Test MAE: 3.067755937576294
Epoch 8, Loss: 5.244282245635986, Accuracy: 2.1211204528808594, Test Loss: 5.171787738800049, Test MAE: 2.124237060546875
Epoch 9, Loss: 5.

In [21]:
model.save('./data/models/mymodel')

INFO:tensorflow:Assets written to: ./data/models/mymodel\assets


In [22]:
loaded_model = keras.models.load_model('./data/models/mymodel')

In [25]:
loaded_model

In [24]:
model

In [26]:
# Let's check:
import numpy as np
np.testing.assert_allclose(
    model.predict(X_test), loaded_model.predict(X_test)
)
%tensorboard --logdir ./data/logs

4/4 [==============================] - 0s 5ms/step


Launching TensorBoard...

In [19]:
%tensorboard --logdir ./data/logs/gradient_tape

Launching TensorBoard...